# py2neo

`py2neo` is one of Neo4j's Python drivers. It offers a fully-featured interface for interacting with your data in Neo4j. Install `py2neo` with `pip install py2neo`.

## Connect

Connect to Neo4j with the `Graph` class.

In [1]:
from py2neo import Graph
from scripts.vis import draw

graph = Graph()

graph.delete_all()

## Nodes

Create nodes with the `Node` class. The first argument is the node's label. The remaining arguments are an arbitrary amount of node properties or key-value pairs.

In [2]:
from py2neo import Node

nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola")
pepsi = Node("Manufacturer", name="Pepsi")

graph.create(nicole, drew, mtdew, cokezero, coke, pepsi)

options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}

draw(graph, options)

## Relationships

Create relationships between nodes with the `Relationship` class.

In [3]:
from py2neo import Relationship

graph.create(Relationship(nicole, "LIKES", cokezero))
graph.create(Relationship(nicole, "LIKES", mtdew))
graph.create(Relationship(drew, "LIKES", mtdew))
graph.create(Relationship(coke, "MAKES", cokezero))
graph.create(Relationship(pepsi, "MAKES", mtdew))

draw(graph, options)

## Cypher

Retrieve Cypher query results with `Graph.cypher.execute`.

In [4]:
query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS name, drink.name AS drink
"""

data = graph.cypher.execute(query)

data

   | name   | drink       
---+--------+--------------
 1 | Nicole | Mountain Dew
 2 | Nicole | Coke Zero   
 3 | Drew   | Mountain Dew

Rows can be accessed by index.

In [12]:
data[0]

 name   | drink       
--------+--------------
 Nicole | Mountain Dew

Columns can be accessed by key or attribute.

In [13]:
for row in data:
    print "{0} likes to drink {1}.".format(row["name"], row.drink)

Nicole likes to drink Mountain Dew.
Nicole likes to drink Coke Zero.
Drew likes to drink Mountain Dew.


## Parameterized Cypher

Pass parameters to Cypher queries by passing additional key-value arguments to `Graph.cypher.execute.` Parameters in Cypher are named and are wrapped in curly braces.

In [14]:
# Average calorie intake of people who know {name}

query = """
MATCH (p1:Person)-[:KNOWS]-(p2:Person)-[:LIKES]->(drink:Drink)
WHERE p1.name = {name}
RETURN p2.name AS name, AVG(drink.calories) AS avg_calories
"""

data = graph.cypher.execute(query, name="Drew")

data

   | name   | avg_calories
---+--------+--------------
 1 | Nicole |       4500.0